In [56]:
import arcgis
from arcgis.gis import GIS
# from datetime import datetime as dt
# from arcgis.geoanalytics.analyze_patterns import calculate_density, find_hot_spots
gis = GIS()

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [39]:
# hybrid : mapa real
# dark-gray-vector : mapa gris
# streets : mapa de calles
# streets-navigation-vector : mapa de calles con navegacion
# streets-night-vector: mapa de calles de noche
m = gis.map(mode='3D')
m.basemap = 'dark-gray-vector'
m

MapView(layout=Layout(height='400px', width='100%'), mode='3D')

In [46]:
cal_density = calculate_density(crime_lyr,
                                weight='Uniform',
                                bin_type='Square',
                                bin_size=1,
                                bin_size_unit="Kilometers",
                                time_step_interval=1,
                                time_step_interval_unit="Years",
                                time_step_repeat_interval=1,
                                time_step_repeat_interval_unit="Months",
                                time_step_reference=dt(2001, 1, 1),
                                radius=1000,
                                radius_unit="Meters",
                                area_units='SquareKilometers',
                                output_name="calculate density of crime" + str(dt.now().microsecond))

<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/World_Countries/FeatureServer/0">